# Inspecting geometry information
We often would like to check the geometry of our simulation: the size of a volume or its material, for example. Information concerning the geometry is saved along the simulation output, in the `geofile`. FairShip itself provides a macro to read this file: $FAIRSHIP/macro/getGeoInformation.py. I have copied the script in this folder to run it together and load the geometry

## Output of the script
Launching it with only the -g option (requiring the geofile), will print only the information of the mother volumes, the one directly below the top volume:

In [6]:
run getGeoInformation.py -g geofile_full.conical.Genie-TGeant4.root

Welcome to JupyROOT 6.18/00
   Detector element             z(midpoint)     halflength       volume-start volume-end   dx                x-start       x-end       dy                y-start       y-end         material
TargetArea_1                : z=-7156.2500cm  dZ=   72.2500cm  [-7228.5000   -7084.0000] dx=  220.0000cm [ -220.0000     220.0000] dy=  225.0000cm [ -225.0000     225.0000]                dummy
rockS_1                     : z=-6813.0000cm  dZ=  271.0000cm  [-7084.0000   -6542.0000] dx= 1500.0000cm [-1500.0000    1500.0000] dy= 1500.0000cm [-1500.0000    1500.0000]             Concrete
MuonShieldArea_1            : z=-5272.0000cm  dZ= 1802.0000cm  [-7074.0000   -3470.0000] dx=  999.9000cm [ -999.9000     999.9000] dy=  999.9000cm [ -999.9000     999.9000]                dummy
floorM_1                    : z=-4250.0000cm  dZ= 2020.9000cm  [-6270.9000   -2229.1000] dx= 1000.0000cm [-1000.0000    1000.0000] dy=  210.0000cm [-1000.0000    -580.0000]             Concrete
tTauNu

We can see the half dimensions, position and material of the volume. We can insect the daughters of a specific volume by adding the option `-v volumname -l daugtherlevel`:

In [7]:
run getGeoInformation.py -g geofile_full.conical.Genie-TGeant4.root -v tTauNuDet_1 -l 1

   Detector element             z(midpoint)     halflength       volume-start volume-end   dx                x-start       x-end       dy                y-start       y-end         material
TargetArea_1                : z=-7156.2500cm  dZ=   72.2500cm  [-7228.5000   -7084.0000] dx=  220.0000cm [ -220.0000     220.0000] dy=  225.0000cm [ -225.0000     225.0000]                dummy
rockS_1                     : z=-6813.0000cm  dZ=  271.0000cm  [-7084.0000   -6542.0000] dx= 1500.0000cm [-1500.0000    1500.0000] dy= 1500.0000cm [-1500.0000    1500.0000]             Concrete
MuonShieldArea_1            : z=-5272.0000cm  dZ= 1802.0000cm  [-7074.0000   -3470.0000] dx=  999.9000cm [ -999.9000     999.9000] dy=  999.9000cm [ -999.9000     999.9000]                dummy
floorM_1                    : z=-4250.0000cm  dZ= 2020.9000cm  [-6270.9000   -2229.1000] dx= 1000.0000cm [-1000.0000    1000.0000] dy=  210.0000cm [-1000.0000    -580.0000]             Concrete
tTauNuDet_1                 : z=-2

Warning in <TGeoManager::Init>: Deleting previous geometry: FAIRGeom/FAIR geometry
Info in <TGeoManager::CloseGeometry>: Geometry loaded from file...
Info in <TGeoManager::SetTopVolume>: Top volume is cave. Master volume is cave
Info in <TGeoManager::Voxelize>: Voxelizing...
Info in <TGeoManager::CountLevels>: max level = 9, max placements = 2000
Info in <TGeoManager::CloseGeometry>: 8469495 nodes/ 1275 volume UID's in FAIR geometry
Info in <TGeoManager::CloseGeometry>: ----------------modeler ready----------------


## Accesing information about a specific volume
Of course, finding a specific volume in this way is quite time-consuming, especially if our desired volume is many levels below the top volume. However, we can directly get our volume by his name thanks to the gGeoManager

In [8]:
volTarget = fGeo.GetVolume("volTarget")
volTarget

<ROOT.TGeoVolume object ("volTarget") at 0x4e4c9e0>

For example, let us ask the half-sizes of our shapes. Remind: different shapes have different size definitions. Vol Target is a simple box, thus we can ask its (DX,DY,DZ) half-dimensions:

In [9]:
dx = volTarget.GetShape().GetDX()
dy = volTarget.GetShape().GetDY()
dz = volTarget.GetShape().GetDZ()
print ("Our target has half-dimensions: ({:.1f},{:.1f},{:.1f}) cm".format(dx,dy,dz))

Our target has half-dimensions: (47.2,73.2,125.7) cm


Another example: often we need the Weight of a volume to predict the number of interactions in it, given the cross section. Let us ask for this quantity:

In [10]:
volTarget.Weight()

7453.881542140798

Info in <TGeoManager::Weight>: Computing analytically weight of volTarget
Info in <TGeoManager::Weight>: Computed weight: 7453.881542 [kg]



As you notice from the printout, the weight is not a member of the volume, which can automatically be called from the class. It is a computation, by looping on daughters and getting density, more information can be found on TGeoVolume reference: https://root.cern.ch/doc/master/classTGeoVolume.html#a4fe233342614a24c755c34edaca5bbd2, along with all the other functions you can use.

For example, we can ask for shape and material properties with a nice printout:

In [11]:
volTarget.InspectShape()

*** Shape TargetBox: TGeoBBox ***
    dX =    47.15750
    dY =    73.24750
    dZ =   125.68500
    origin: x=    0.00000 y=    0.00000 z=    0.00000


In [12]:
volTarget.InspectMaterial()

Mixture air    Aeff=14.8329 Zeff=7.385 rho=0.001205 radlen=30245.4 intlen=71036.3 index=0
   Element #0 : N  Z=  7.00 A= 14.01 w= 0.755
   Element #1 : O  Z=  8.00 A= 16.00 w= 0.231
   Element #2 : AR  Z= 18.00 A= 39.95 w= 0.014


In [13]:
fGeo.Export("exporttest.root")

3144984

Info in <TGeoManager::Export>: Exporting FAIRGeom FAIR geometry as root file. Optimizations streamed.
